In [1]:
import pandas as pd
import numpy as np
import flask
import json as js
#import pyproj as pp

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///Hurricane.sqlite', echo=False)
con = engine.connect()

In [32]:
con.execute('''CREATE TABLE Hurricanes(
                ID integer not null primary key, 
                Name Varchar (100),
                Time integer not null,
                Event Varchar (100),
                Status Varchar (100),
                Latitude integer not null,
                Longitude integer not null,
                Wind integer not null,
                Pressure integer not null,
                ISODate Varchar (100),
                Location Varchar (100)
                )''')


IndentationError: unexpected indent (<ipython-input-32-f8af27f2f61c>, line 2)

In [4]:
df = pd.read_csv('hurdat.csv', index_col=0)
df.to_sql('Hurricanes', engine, if_exists='replace', index=True, index_label="id")

In [5]:
data = pd.read_sql('Select * from Hurricanes',engine)


In [6]:
data.head()

,id,Name,Date,Time,Event,Status,Latitude,Longitude,Wind,Pressure,ISODate,Location
0,EP011949,UNNAMED,19490611,0,,TS,20.2,-106.3,45,-999,1949-06-11T00:00:00,East Pacific
1,EP011949,UNNAMED,19490611,600,,TS,20.2,-106.4,45,-999,1949-06-11T06:00:00,East Pacific
2,EP011949,UNNAMED,19490611,1200,,TS,20.2,-106.7,45,-999,1949-06-11T12:00:00,East Pacific
3,EP011949,UNNAMED,19490611,1800,,TS,20.3,-107.7,45,-999,1949-06-11T18:00:00,East Pacific
4,EP011949,UNNAMED,19490612,0,,TS,20.4,-108.6,45,-999,1949-06-12T00:00:00,East Pacific


In [35]:
### function takes an ID parameter, gets data from database, converts to and returns geoJson 
## requires geojson to be installed ( pip install geojson )
## requires imports 
from geojson import LineString, Feature, Point, GeometryCollection, FeatureCollection
def makeGeo(id):
    edf=  pd.read_sql('Select * from Hurricanes where id="'+id+'" Order by Date,Time',engine)
    ls=LineString(edf[['Latitude','Longitude']].values.tolist())
    dates=edf['ISODate'].values.tolist()
    winds=edf['Wind'].values.tolist()
    name= edf['Name'].unique()[0].strip() 
    f=Feature(geometry=ls,properties={"id":"AL041966", "name":name , "dates":dates, "winds":winds})
    fc=FeatureCollection([f])
    return fc


In [36]:
fc=makeGeo('AL041966')
fc

{"features": [{"geometry": {"coordinates": [[31.0, -41.0], [31.5, -41.5], [31.7, -41.7], [31.8, -41.9], [32.0, -42.0], [32.2, -42.0], [32.4, -42.1], [32.7, -42.2], [33.0, -42.1], [33.3, -41.8], [33.8, -41.1], [34.5, -40.5], [35.3, -40.5], [35.9, -40.8], [36.1, -41.4], [36.2, -41.9], [36.4, -42.2], [36.6, -42.2], [37.4, -41.9], [38.3, -41.4], [38.7, -40.8], [39.2, -40.1], [40.1, -39.2], [40.9, -38.5], [41.6, -38.2], [42.4, -38.0], [43.1, -37.7], [43.9, -37.4], [44.6, -37.4], [45.3, -37.4], [46.0, -37.3], [46.8, -37.1], [47.7, -36.4], [49.0, -36.0], [50.5, -36.5], [52.0, -37.0], [53.5, -38.5]], "type": "LineString"}, "properties": {"dates": ["1966-07-22T18:00:00", "1966-07-23T00:00:00", "1966-07-23T06:00:00", "1966-07-23T12:00:00", "1966-07-23T18:00:00", "1966-07-24T00:00:00", "1966-07-24T06:00:00", "1966-07-24T12:00:00", "1966-07-24T18:00:00", "1966-07-25T00:00:00", "1966-07-25T06:00:00", "1966-07-25T12:00:00", "1966-07-25T18:00:00", "1966-07-26T00:00:00", "1966-07-26T06:00:00", "1966-0